In [1]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
from env import key, client
import datetime
from datetime import date
from datetime import datetime, timedelta
from dateutil.relativedelta import *

import json

import climacell_api 

In [2]:
# This request is the San Antonio Botanical Garden. It returns realtime data for the identified fields.
response = client.realtime(lat= 29.4576, lon=98.4587, 
                           fields=['precipitation', 'temp', 'humidity', 'wind_speed', 'cloud_cover'])
# Returns status code (Query Successful or Not)
print(response.status_code)
# Create df of results to visualize
data = response.json()
df = pd.DataFrame(data)
df.head()

200


,lat,lon,temp,wind_speed,humidity,precipitation,cloud_cover,observation_time
value,29.4576,98.4587,-2.56,1.88,49.81,0,0,2021-01-22T21:12:41.616Z
units,29.4576,98.4587,C,m/s,%,mm/hr,%,NaN


In [3]:
#from climacell_api import ClimacellApiClient
r = client.historical_station(lat=40, lon=50, start_time='2021-01-22T20:47:00Z', end_time='now', fields=['temp', 'wind_gust'])
r.status_code

200

## Historical Climacell
`Data returned is a list of forecast data for a specific location up to 6 hours in the past.`

In [4]:
stop = datetime.now().isoformat()
start = (datetime.now() - timedelta(hours=6)).isoformat()

url = "https://api.climacell.co/v3/weather/historical/climacell"

querystring = {'lat': 40, 
               'lon': 50,
               "timestep":30,
               "unit_system":"si",
               "apikey":key, 
               "start_time": start, 
               "end_time": stop }

response = requests.request("GET", url, params=querystring)
data = response.json()
df_clima = pd.DataFrame(data)
df_clima.head(5)

,lat,lon,precipitation,precipitation_type,observation_time
0,40,50,"{'value': None, 'units': 'mm/hr'}",{'value': None},{'value': '2021-01-22T09:12:41.966Z'}
1,40,50,"{'value': None, 'units': 'mm/hr'}",{'value': None},{'value': '2021-01-22T09:42:41.966Z'}
2,40,50,"{'value': None, 'units': 'mm/hr'}",{'value': None},{'value': '2021-01-22T10:12:41.966Z'}
3,40,50,"{'value': None, 'units': 'mm/hr'}",{'value': None},{'value': '2021-01-22T10:42:41.966Z'}
4,40,50,"{'value': None, 'units': 'mm/hr'}",{'value': None},{'value': '2021-01-22T11:12:41.966Z'}


## Historical Station
`24 hours in the past'

In [13]:
stop = datetime.now().isoformat()
start = (datetime.now() - relativedelta(days=1)).isoformat()

url = "https://api.climacell.co/v3/weather/historical/station"

querystring = {'lat': 30, 
               'lon': 40,
               "unit_system":"si",
               "apikey":key,
               "start_time": start, 
               "end_time": stop,
               "fields":['temp','precipitation','humidity','baro_pressure'],
              }


response = requests.request("GET", url, params=querystring)
data = response.json()
df_hist = pd.DataFrame(data)
df_hist.head(5)
#df_hist.to_csv('climate_data_csv')

### Unpack Column and turn it in to individual colums in data frame

In [6]:
temp = df_hist['temp'].apply(pd.Series)
temp.rename(columns={'value':'temp_in_c'}, inplace = True)
temp.drop(columns=['units'], inplace = True)
temp_df = pd.DataFrame(temp)
temp_df

,temp_in_c
0,8
1,7
2,6
3,6
4,5
5,4
6,4
7,4
8,3
9,3


### Append to DF

In [7]:
result = pd.concat([df_hist, temp], axis=1)
result

,temp,baro_pressure,humidity,precipitation,observation_time,temp_in_c
0,"{'value': 8, 'units': 'C'}","{'value': 1031, 'units': 'hPa'}","{'value': None, 'units': '%'}","{'value': None, 'units': 'mm/hr'}",{'value': '2021-01-21T16:00:00.000Z'},8
1,"{'value': 7, 'units': 'C'}","{'value': 1032, 'units': 'hPa'}","{'value': None, 'units': '%'}","{'value': None, 'units': 'mm/hr'}",{'value': '2021-01-21T17:00:00.000Z'},7
2,"{'value': 6, 'units': 'C'}","{'value': 1032, 'units': 'hPa'}","{'value': None, 'units': '%'}","{'value': None, 'units': 'mm/hr'}",{'value': '2021-01-21T18:00:00.000Z'},6
3,"{'value': 6, 'units': 'C'}","{'value': 1032, 'units': 'hPa'}","{'value': None, 'units': '%'}","{'value': None, 'units': 'mm/hr'}",{'value': '2021-01-21T19:00:00.000Z'},6
4,"{'value': 5, 'units': 'C'}","{'value': 1032, 'units': 'hPa'}","{'value': None, 'units': '%'}","{'value': None, 'units': 'mm/hr'}",{'value': '2021-01-21T20:00:00.000Z'},5
5,"{'value': 4, 'units': 'C'}","{'value': 1032, 'units': 'hPa'}","{'value': None, 'units': '%'}","{'value': None, 'units': 'mm/hr'}",{'value': '2021-01-21T21:00:00.000Z'},4
6,"{'value': 4, 'units': 'C'}","{'value': 1031, 'units': 'hPa'}","{'value': None, 'units': '%'}","{'value': None, 'units': 'mm/hr'}",{'value': '2021-01-21T22:00:00.000Z'},4
7,"{'value': 4, 'units': 'C'}","{'value': 1031, 'units': 'hPa'}","{'value': None, 'units': '%'}","{'value': None, 'units': 'mm/hr'}",{'value': '2021-01-21T23:00:00.000Z'},4
8,"{'value': 3, 'units': 'C'}","{'value': 1031, 'units': 'hPa'}","{'value': None, 'units': '%'}","{'value': None, 'units': 'mm/hr'}",{'value': '2021-01-22T00:00:00.000Z'},3
9,"{'value': 3, 'units': 'C'}","{'value': 1031, 'units': 'hPa'}","{'value': None, 'units': '%'}","{'value': None, 'units': 'mm/hr'}",{'value': '2021-01-22T01:00:00.000Z'},3


In [8]:
now = datetime.now()
year = '{:02d}'.format(now.year)
month = '{:02d}'.format(now.month)
day = '{:02d}'.format(now.day)
hour = '{:02d}'.format(now.hour)
minute = '{:02d}'.format(now.minute)
day_month_year_hour_now = '{}-{}-{}-{}'.format(year, month, day, hour)
day_month_year_hour_now

'2021-01-22-15'

In [9]:
six_hour_ago = datetime.now() - timedelta(hours=6)
year = '{:02d}'.format(six_hour_ago.year)
month = '{:02d}'.format(six_hour_ago.month)
day = '{:02d}'.format(six_hour_ago.day)
hour = '{:02d}'.format(six_hour_ago.hour)
minute = '{:02d}'.format(six_hour_ago.minute)
day_month_year_hour_past = '{}-{}-{}-{}'.format(year, month, day, hour)
day_month_year_hour_past

'2021-01-22-09'

In [10]:
start = datetime.now().isoformat()
start

'2021-01-22T15:12:42.701145'

In [11]:
stop = (datetime.now() - relativedelta(months=4)).isoformat()
stop

'2020-09-22T15:12:42.709652'